# LRE on Hardware

## Executor and Backend Setup

First import all required libraries and modules.

In [1]:
import json
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import numpy as np

from functools import partial
import qiskit
from qiskit_aer import QasmSimulator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

import cache_utils

from mitiq import lre
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
from mitiq.lre.multivariate_scaling.layerwise_folding import _get_num_layers_without_measurements

Define an executor 

In [2]:
def ibmq_executor(circuit: qiskit.QuantumCircuit, backend, shots: int = 100) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run (assumes no measurements).
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        basis_gates=None,
        optimization_level=0,  # Important to preserve folded gates.
    )
    exec_circuit.measure_active()

    num_qubits = circuit.num_qubits

    # Run the circuit
    sampler = Sampler(mode=backend)
    job = sampler.run([exec_circuit], shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result()[0].data.measure.get_counts()
    ground_state = "0" * num_qubits
    expectation_value = counts.get(ground_state, 0.0) / shots
    return expectation_value

In [3]:
#################################################
# HARDWARE
#################################################
service = QiskitRuntimeService()
# hard_backend = service.least_busy(operational=True, simulator=False)
hard_backend = service.backend("ibm_sherbrooke")
print(hard_backend)
hardware_executor = partial(ibmq_executor, backend=hard_backend)

#################################################
# SIMULATOR
#################################################
noise_model = NoiseModel.from_backend(hard_backend)
sim_backend = AerSimulator.from_backend(hard_backend)

# old_sim_backend = QasmSimulator(method="density_matrix", noise_model=noise_model)
# new_sim_backend = AerSimulator(method="density_matrix", noise_model=noise_model)

sim_backend_ideal = QasmSimulator()
simulator_executor = partial(ibmq_executor, backend=sim_backend)

<IBMBackend('ibm_sherbrooke')>


## Hyperparameter Tuning

First step is to do tests on simulators for hyperparamter tuning. With LRE, the primary paramters that can be adjusted are `degree` and `fold_multiplier`. Due to performance, it has been noted that `num_chunks` will also be critical for testing.

### Manual Example for testing

In [4]:
num_qubits = 4 # 9, 16, 25, 36, 49 (available through benchpress)
qasm_filename = f"square-heisenberg-{num_qubits}"
# qasm_filename = f"qft-{num_qubits}"
circuit = qiskit.qasm2.load(f"{qasm_filename}.qasm")

CIRCUIT_NAME = qasm_filename
DEGREE = 3
FOLD_MULTIPLIER = 2
NUM_CHUNKS = 0

ideal = ibmq_executor(circuit, sim_backend_ideal)
unmitigated = ibmq_executor(circuit, sim_backend)
# new_unmitigated = ibmq_executor(circuit, new_sim_backend)
# old_unmitigated = ibmq_executor(circuit, old_sim_backend)



mitigated = lre.execute_with_lre(
    circuit, simulator_executor, degree=1, fold_multiplier=1, num_chunks=6
)


print(f"ideal={ideal:.3f} unmitigated={unmitigated:.3f}")
# print(f"ideal={ideal:.3f} unmitigated={unmitigated:.3f} mitigiated={mitigated:.3f}")

#################################################
# save ideal(time = 0.0) and unmitigated(time = 1.0)
#################################################
ideal_res = [ qasm_filename, sim_backend.name, ideal, 0, 0, 0, 0]
unmitigated_res = [ qasm_filename, sim_backend.name, unmitigated, 0, 0, 0, 1]
# cache_utils.save_result(ideal_res)
# cache_utils.save_result(unmitigated_res)


ideal=1.000 unmitigated=0.710


In [6]:
mitigated = lre.execute_with_lre(
    circuit, simulator_executor, degree=1, fold_multiplier=1, num_chunks=7
)

In [ ]:
df = pd.read_csv('results/cached_results.csv')
df = df.sort_values(by=["circuit", "backend", "fold_multiplier"])
df

In [ ]:
cache_res = [
                   qasm_filename,
                   sim_backend.name,
                   np.nan,
                   0,
                   0,
                   0,
                   99999 
                ]
cache_utils.save_result(cache_res)

In [ ]:
print(circuit)
print(len(circuit))

### Automated Example with Saving Results

First setup the parameters to test including: degree, fold_multiplier, and num_chunks.

In [7]:
MAX_DEGREE = 10
MAX_FOLD_MULTIPLIER = 2 
MAX_NUM_CHUNKS = 10 

# num_qubits = [4, 9, 16, 25, 36, 49]
num_qubits = [4]
qasm_filenames = [f"square-heisenberg-{num_qubit}" for num_qubit in num_qubits]
# qasm_filenames = [f"qft-{num_qubit}" for num_qubit in num_qubits]

# Setup all of the different values/circuits to test
circuits = [qiskit.qasm2.load(f"{qasm_filename}.qasm") for qasm_filename in qasm_filenames]
degrees = list(range(1,MAX_DEGREE+1))
fold_multipliers = list(range(1, MAX_FOLD_MULTIPLIER+1))
num_chunks = list(range(1, MAX_NUM_CHUNKS+1))


Run experiments on a noisy simulator to get results and store them into a results/cached_results.csv for later analysis.

In [ ]:
for i in range(len(circuits)):
    circuit = circuits[i]
    for fold_multiplier in fold_multipliers:
        for degree in degrees:
            for num_chunk in num_chunks:
                if num_chunk >= len(circuit):
                    print(f"NUM_CHUNKS: fold={fold_multiplier} degree={degree} num_chunk={num_chunk} lenght={len(circuit)}")
                    break
                t0 = time.time()
                try:
                    res = lre.execute_with_lre(circuit, simulator_executor, degree=degree, fold_multiplier=fold_multiplier, num_chunks=num_chunk)
                except RuntimeWarning as w:
                    print(f"RUNTIME degree={degree} fold_multiplier={fold_multiplier} num_chunks={num_chunks}: {w}")
                    break
                except ValueError as e:
                    print(f"VALUEERROR: {e}")
                    if "Determinant of sample matrix cannot be calculated as the matrix is too large" in str(e):
                        t0 = time.time()-t0
                        cache_res = [
                            qasm_filenames[i],
                            sim_backend.name,
                            np.nan,
                            degree,
                            fold_multiplier,
                            num_chunk,
                            t0
                        ]
                        cache_utils.save_result(cache_res)

                        break  # Go to the next iteration of the loop
                    else:  # Re-raise if it's a different ValueError
                        raise  # Important: Don't silently ignore other ValueErrors!
                except Exception as e: #Catch other exceptions
                    print(f"EXCEPTION {e}")
                    raise #Reraise the exception

                t0 = time.time()-t0
                cache_res = [
                   qasm_filenames[i],
                   sim_backend.name,
                   res,
                   degree,
                   fold_multiplier,
                   num_chunk,
                   t0
                ]
                cache_utils.save_result(cache_res)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1204b5dc0>>
Traceback (most recent call last):
  File "/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
# Read from CSV results
file_name = 'results/cached_results.csv'
df = pd.read_csv(file_name)
df.sort_values(by=["circuit, result"])
squareh_4 = df.loc[(df["circuit"] == "square-heisenberg-4") & (df["backend"] == "aer_simulator_from(ibm_sherbrooke)")]
degrees = squareh_4["degree"]
fold_multipliers = squareh_4["fold_multiplier"]
num_chunks = squareh_4["num_chunks"]
results = squareh_4["result"]

plt.figure(figsize=(8,6))
scatter = plt.scatter(degrees, fold_multipliers, c=num_chunks, cmap="viridis", s = (1-results)*100)

plt.xlabel("degree")
plt.ylabel("fold_multiplier")
plt.title("Hyperparameter tuning for LRE for square-heisenberg-4")
plt.colorbar(scatter, label="num_chunks")

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
squareh_4.loc[squareh_4["result"] = ideal - squareh_4["result"]

In [ ]:
squareh_4